In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ***Why Modin?***

We use Modin, (developed by Intel and part of One-MKL): The drop-in replacement for Pandas which provided several benefits, like
Reduced Memory Usage
Improved Performance 
Scalability

In [ ]:
!pip install modin[all]

In [ ]:
#import pandas as pd
import modin.pandas as pd

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Intel Challenge/test_data_sentiment.csv")

In [ ]:
df.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn
0,BJ8bLZcYcm,How Millennials Are Changing The Global Economy,But though millennials may have been born into...,Seeking Alpha,economy,17-02-2016 19:09,6,0,3
1,ph8TGKbvNp,China Promises It Will Keep Its Economy Strong,"""""""China's economic fundamentals have not chan...",Fortune,economy,17-02-2016 19:10,5,1,1
2,qJDgUHX7Ay,Fed eyes increased risks for US economy,Many Federal Reserve policymakers saw increase...,Financial Times,economy,17-02-2016 19:10,45,2,40
3,2MVdzsl9s1,Obama to visit Cuba next month: reports,US President Barack Obama will make a historic...,AFP via Yahoo! News,obama,17-02-2016 19:12,17,0,0
4,6QFCqKq50Q,White House: Obama Will Not Attend Justice Sca...,President Barack Obama will not attend Justice...,NBCNews.com,obama,17-02-2016 19:12,5568,181,36


In [ ]:
import numpy as np

In [ ]:
headline = list(df['Headline'])
title = list(df['Title'])

In [ ]:
print(title[1])
print(headline[1])

In [ ]:
print(len(title))
print(len(headline))


15424

# **Convert words of the sentence into features using Word2Vec** 
 Word embeddings is a technique where individual words are transformed into a numerical representation of the word (a vector). Where each word is mapped to one vector, this vector is then learned in a way which resembles a neural network.The effectiveness of Word2Vec comes from its ability to group together vectors of similar words. Given a large enough dataset, Word2Vec can make strong estimates about a word’s meaning based on their occurrences in the text.

In [ ]:
import gensim.downloader as api

info = api.info()  # show info about available models/datasets
model = api.load("word2vec-google-news-300") 

[=================================================-] 99.9% 1661.6/1662.8MB downloaded


In [ ]:
import nltk
nltk.download('punkt')

title_words = [ nltk.word_tokenize(sent.lower()) for sent in title]

headline_words = [nltk.word_tokenize(sent.lower()) for sent in headline]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# ***Remove Stopwords***

**From title**

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

Sem_Words_title=[]
Sub_List_title=[]
for sent in title_words:
  for word in sent:
    if word not in stop_words and word.isalnum():
      Sub_List_title.append(word)
  Sem_Words_title.append(Sub_List_title)
  Sub_List_title=[]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**From Heading**

In [ ]:
Sem_Words_head=[]
Sub_List_head=[]
for sent in headline_words:
  for word in sent:
    if word not in stop_words and word.isalnum():
      Sub_List_head.append(word)
  Sem_Words_head.append(Sub_List_head)
  Sub_List_head=[]

# ***Create Sentence embedding from word embeddings*** - 





**From Title**

In [ ]:
word_vectors_mat_title=[]
for sent in Sem_Words_title:
  i=0
  if len(sent)>18 :
   word_vectors = [model[sent[i]] for i in range(18) if sent[i] in model.key_to_index]
  else:
   word_vectors = [model[words] for words in sent if words in model.key_to_index]
  word_vectors_mat_title.append(word_vectors)
  word_vectors=[]

In [ ]:
print(len(word_vectors_mat_title[1][1]))

300


In [ ]:
sentence_embedding_mat_title=[]
for word_vectors in word_vectors_mat_title:
  
  sentence_embedding = [np.mean(word_vectors) for words in word_vectors]
  sentence_embedding_mat_title.append(sentence_embedding)

sentence_embedding_mat_title=np.array(sentence_embedding_mat)

**From Headline**

In [ ]:
word_vectors_mat_head=[]
for sent in Sem_Words_head:
  i=0
  if len(sent)>48 :
   word_vectors = [model[sent[i]] for i in range(48) if sent[i] in model.key_to_index]
  else:
   word_vectors = [model[words] for words in sent if words in model.key_to_index]
  word_vectors_mat_head.append(word_vectors)
  word_vectors=[]

In [ ]:
sentence_embedding_mat_head=[]
for word_vectors in word_vectors_mat_head:
  
  sentence_embedding = [np.mean(word_vectors) for words in word_vectors]
  sentence_embedding_mat_head.append(sentence_embedding)

sentence_embedding_mat_head=np.array(sentence_embedding_mat)

In [ ]:
print(sentence_embedding_mat_head[15423])

In [ ]:
print(sentence_embedding_mat_head.shape)

In [ ]:
max_length_headline = 48
max_length_title = 18

# ***Final Sentence Embedding Matrix Preparation***

In [ ]:
final_sent_mat_title = []
final_sent_mat_head = []

**For Title**

In [ ]:
i=0
for i in range(len(sentence_embedding_mat_title)):
  sentence_embedding = list(sentence_embedding_mat_title[i])
  sent_size = len(sentence_embedding)
  if max_length_title-sent_size > 0:
   sentence_embedding = sentence_embedding + [0]*(max_length_title - sent_size) 
  final_sent_mat_title.append(sentence_embedding) 

In [ ]:
final_sent_mat_title = np.array(final_sent_mat_title)
final_sent_mat_title = final_sent_mat_title.astype('float64')


In [ ]:
final_sent_mat_title.shape

(15424, 48)

In [ ]:
df_test_title={"ID_Link":list(df['IDLink']),"Test_Title":list(final_sent_mat_title)}
df_test_headline = {"ID_Link":list(df['IDLink']),"Test_Headline":list(final_sent_mat_head)}

# **Saving the final preprocessed data for the headline and title**

In [ ]:
df_final_test_title = pd.DataFrame.from_dict(df_test_title)

In [ ]:
df_final_test_head = pd.DataFrame.from_dict(df_test_headline)

# ***Visualizing the Preprocessed Data***

In [ ]:
df_final_test_head.head()

,ID_Link,Test_Headline
0,BJ8bLZcYcm,"[0.00039006926817819476, 0.0003900692681781947..."
1,ph8TGKbvNp,"[0.00402815779671073, 0.00402815779671073, 0.0..."
2,qJDgUHX7Ay,"[-0.005776678677648306, -0.005776678677648306,..."
3,2MVdzsl9s1,"[0.003517178585752845, 0.003517178585752845, 0..."
4,6QFCqKq50Q,"[0.0019788655918091536, 0.0019788655918091536,..."


In [ ]:
df_final_test_title.to_csv("/content/drive/MyDrive/Intel Challenge/Final_Test_data/Test_Title.csv", index = False)
df_final_test_head.to_csv("/content/drive/MyDrive/Intel Challenge/Final_Test_data/Test_Headline.csv", index = False)